In [24]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math

In [25]:
# Load the competition datasets into Pandas DataFrame
train_proteins = pd.read_csv("~/Documents/amp-parkinsons-disease-progression-prediction/train_proteins.csv")
train_peptides = pd.read_csv("~/Documents/amp-parkinsons-disease-progression-prediction/train_peptides.csv")
train_clinical = pd.read_csv("~/Documents/amp-parkinsons-disease-progression-prediction/train_clinical_data.csv")
supplemental_clinical = pd.read_csv("~/Documents/amp-parkinsons-disease-progression-prediction/supplemental_clinical_data.csv")

In [26]:
print(train_clinical.shape)
print(train_peptides.shape)
print(train_proteins.shape)
print(supplemental_clinical.shape)

(2615, 8)
(981834, 6)
(232741, 5)
(2223, 8)


In [27]:
train_clinical.head(10)

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication
0,55_0,55,0,10.0,6.0,15.0,NaN,NaN
1,55_3,55,3,10.0,7.0,25.0,NaN,NaN
2,55_6,55,6,8.0,10.0,34.0,NaN,NaN
3,55_9,55,9,8.0,9.0,30.0,0.0,On
4,55_12,55,12,10.0,10.0,41.0,0.0,On
5,55_18,55,18,7.0,13.0,38.0,0.0,On
6,55_24,55,24,16.0,9.0,49.0,0.0,On
7,55_30,55,30,14.0,13.0,49.0,0.0,On
8,55_36,55,36,17.0,18.0,51.0,0.0,On
9,55_42,55,42,12.0,20.0,41.0,0.0,On


In [28]:
train_peptides.head()

,visit_id,visit_month,patient_id,UniProt,Peptide,PeptideAbundance
0,55_0,0,55,O00391,NEQEQPLGQWHLS,11254.3
1,55_0,0,55,O00533,GNPEPTFSWTK,102060.0
2,55_0,0,55,O00533,IEIPSSVQQVPTIIK,174185.0
3,55_0,0,55,O00533,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,27278.9
4,55_0,0,55,O00533,SMEQNGPGLEYR,30838.7


In [29]:
train_proteins.head()

,visit_id,visit_month,patient_id,UniProt,NPX
0,55_0,0,55,O00391,11254.3
1,55_0,0,55,O00533,732430.0
2,55_0,0,55,O00584,39585.8
3,55_0,0,55,O14498,41526.9
4,55_0,0,55,O14773,31238.0


In [30]:
supplemental_clinical.head()

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication
0,35_0,35,0,5.0,3.0,16.0,0.0,NaN
1,35_36,35,36,6.0,4.0,20.0,0.0,NaN
2,75_0,75,0,4.0,6.0,26.0,0.0,NaN
3,75_36,75,36,1.0,8.0,38.0,0.0,On
4,155_0,155,0,NaN,NaN,0.0,NaN,NaN


In [31]:
# Determining the number of unique patients in the datasets.

patient_count = train_clinical['patient_id'].nunique()
print(f'There are {patient_count} number of unique patients in "train_clinical_data.csv"')

patient_count1 = train_peptides['patient_id'].nunique()
print(f'There are {patient_count1} number of unique patients in "train_peptides.csv"')

patient_count2 = train_proteins['patient_id'].nunique()
print(f'There are {patient_count2} number of unique patients in "train_proteins.csv"')
      
patient_count3 = supplemental_clinical['patient_id'].nunique()
print(f'There are {patient_count3} number of unique patients in "supplemental_clinical_data.csv"')

There are 248 number of unique patients in "train_clinical_data.csv"
There are 248 number of unique patients in "train_peptides.csv"
There are 248 number of unique patients in "train_proteins.csv"
There are 771 number of unique patients in "supplemental_clinical_data.csv"


In [35]:
# list all the unique patient_id values. 
train_clinical['patient_id'].unique()

[   55   942  1517  1923  2660  3636  3863  4161  4172  4923  5027  5036
  5178  5645  5742  6054  6211  6420  7051  7117  7151  7265  7508  7568
  7832  7886  8344  8699 10053 10138 10174 10541 10715 10718 11459 11686
 11928 12516 12636 12703 12755 12931 13360 13368 13618 13804 13852 13968
 14035 14124 14242 14270 14344 14450 14811 15009 15245 15504 15590 16238
 16347 16566 16574 16778 16931 17154 17201 17414 17727 18183 18204 18553
 18560 19088 20212 20216 20352 20404 20460 20581 20664 20707 20791 20792
 21126 21537 21729 22126 22623 23175 23192 23244 23391 23636 24278 24690
 24818 24820 24911 25562 25739 25750 25827 25911 26005 26104 26210 26809
 27079 27300 27464 27468 27607 27715 27872 27893 27971 27987 28327 28342
 28818 29313 29417 30119 30155 30416 30894 30951 31121 31154 31270 31693
 33108 33548 33558 34182 35231 35465 35477 35675 35696 36797 37220 37312
 37566 38419 39144 39719 40022 40200 40340 40650 40751 40798 40874 40876
 40967 40980 41444 41617 41628 41871 41883 41930 42

In [33]:
# Look at the complete clincal data for one patient: patient_id = 55
train_clinical[train_clinical['patient_id']==55]

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication
0,55_0,55,0,10.0,6.0,15.0,NaN,NaN
1,55_3,55,3,10.0,7.0,25.0,NaN,NaN
2,55_6,55,6,8.0,10.0,34.0,NaN,NaN
3,55_9,55,9,8.0,9.0,30.0,0.0,On
4,55_12,55,12,10.0,10.0,41.0,0.0,On
5,55_18,55,18,7.0,13.0,38.0,0.0,On
6,55_24,55,24,16.0,9.0,49.0,0.0,On
7,55_30,55,30,14.0,13.0,49.0,0.0,On
8,55_36,55,36,17.0,18.0,51.0,0.0,On
9,55_42,55,42,12.0,20.0,41.0,0.0,On


In [36]:
# Look at the complete clincal data for another patient: patient_id = 942
train_clinical[train_clinical['patient_id']==942]

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication
13,942_0,942,0,3.0,2.0,20.0,NaN,NaN
14,942_3,942,3,7.0,2.0,17.0,NaN,NaN
15,942_6,942,6,8.0,2.0,21.0,NaN,NaN
16,942_12,942,12,5.0,2.0,25.0,0.0,NaN
17,942_18,942,18,6.0,5.0,18.0,0.0,NaN
18,942_24,942,24,2.0,3.0,23.0,NaN,NaN
19,942_30,942,30,4.0,4.0,19.0,0.0,NaN
20,942_36,942,36,2.0,4.0,19.0,0.0,NaN
21,942_42,942,42,2.0,3.0,32.0,0.0,NaN
22,942_48,942,48,2.0,6.0,35.0,0.0,NaN


In [60]:
# Look at the complete clincal data for another patient: patient_id = 1517
train_clinical[train_clinical['patient_id']==1517]

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication
28,1517_0,1517,0,11.0,6.0,25.0,5.0,On
29,1517_6,1517,6,17.0,4.0,30.0,7.0,On
30,1517_12,1517,12,20.0,11.0,32.0,5.0,On
31,1517_18,1517,18,23.0,14.0,25.0,5.0,On
32,1517_24,1517,24,19.0,11.0,28.0,3.0,On
33,1517_30,1517,30,20.0,15.0,39.0,10.0,Off
34,1517_36,1517,36,20.0,17.0,31.0,8.0,On
35,1517_42,1517,42,15.0,13.0,37.0,4.0,On
36,1517_48,1517,48,20.0,18.0,40.0,6.0,Off
37,1517_60,1517,60,19.0,18.0,39.0,10.0,Off


In [58]:
# I notice a lot of NaN values in the previous patient data, especially in 'clinical_state_on_medication' column,
# so let's count all the NaN values in each column.

train_clinical.isna().sum()

visit_id                                  0
patient_id                                0
visit_month                               0
updrs_1                                   1
updrs_2                                   2
updrs_3                                  25
updrs_4                                1038
upd23b_clinical_state_on_medication    1327
dtype: int64

In [57]:
# counting the number of unique scores in 'updrs_4' and their occurances.
train_clinical['updrs_4'].value_counts()

0.0     961
3.0     107
4.0      86
2.0      83
5.0      82
1.0      64
6.0      55
7.0      40
8.0      27
9.0      22
10.0     17
11.0      8
13.0      7
14.0      7
12.0      5
16.0      3
20.0      1
15.0      1
17.0      1
Name: updrs_4, dtype: int64

In [59]:
# counting the number of unique scores for 'clinical state on medication' and their occurances.

train_clinical['upd23b_clinical_state_on_medication'].value_counts()

On     775
Off    513
Name: upd23b_clinical_state_on_medication, dtype: int64

In [ ]:
# I decide to drop the "medication status" column (due to over 50% NaN values).

train_clinical.drop('upd23b_clinical_state_on_medication', axis=1, inplace=True)


In [64]:
train_clinical

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4
0,55_0,55,0,10.0,6.0,15.0,NaN
1,55_3,55,3,10.0,7.0,25.0,NaN
2,55_6,55,6,8.0,10.0,34.0,NaN
3,55_9,55,9,8.0,9.0,30.0,0.0
4,55_12,55,12,10.0,10.0,41.0,0.0
...,...,...,...,...,...,...,...
2610,65043_48,65043,48,7.0,6.0,13.0,0.0
2611,65043_54,65043,54,4.0,8.0,11.0,1.0
2612,65043_60,65043,60,6.0,6.0,16.0,1.0
2613,65043_72,65043,72,3.0,9.0,14.0,1.0


In [71]:
# and use interpolate() on the rest of the dataset.

train_clinical['updrs_1'].interpolate(method='linear', limit_direction='both', inplace=True)
train_clinical['updrs_2'].interpolate(method='linear', limit_direction='both', inplace=True)
train_clinical['updrs_3'].interpolate(method='linear', limit_direction='both', inplace=True)
train_clinical['updrs_4'].interpolate(method='linear', limit_direction='both', inplace=True)

In [72]:
# no more NaN values remaining. 

train_clinical.isna().sum()

visit_id       0
patient_id     0
visit_month    0
updrs_1        0
updrs_2        0
updrs_3        0
updrs_4        0
dtype: int64

In [73]:
# let's look at the other datasets. 

train_peptides.isnull().sum()

visit_id            0
visit_month         0
patient_id          0
UniProt             0
Peptide             0
PeptideAbundance    0
dtype: int64

In [74]:
train_proteins.isnull().sum()

visit_id       0
visit_month    0
patient_id     0
UniProt        0
NPX            0
dtype: int64

In [75]:
supplemental_clinical.isnull().sum()

visit_id                                  0
patient_id                                0
visit_month                               0
updrs_1                                 213
updrs_2                                 214
updrs_3                                   5
updrs_4                                 928
upd23b_clinical_state_on_medication    1101
dtype: int64

In [ ]:
# we'll also drop the 'medication' column and interpolate the rest using the same method.
supplemental_clinical.drop('upd23b_clinical_state_on_medication', axis=1, inplace=True)


In [80]:
supplemental_clinical['updrs_1'].interpolate(method='linear', limit_direction='both', inplace=True)
supplemental_clinical['updrs_2'].interpolate(method='linear', limit_direction='both', inplace=True)
supplemental_clinical['updrs_3'].interpolate(method='linear', limit_direction='both', inplace=True)
supplemental_clinical['updrs_4'].interpolate(method='linear', limit_direction='both', inplace=True)

In [81]:
supplemental_clinical.isnull().sum()

visit_id       0
patient_id     0
visit_month    0
updrs_1        0
updrs_2        0
updrs_3        0
updrs_4        0
dtype: int64